# FLEKS Python Visualization Toolkit

flekspy is a Python package for processing FLEKS data.

## FLEKS data format

* Field: *.out format or AMREX built-in format, whose directory name is assumed to end with "_amrex"
* PIC particle: AMREX built-in format 
* Test particle: binary data format

## Importing the package

In [ ]:
import flekspy

## Downloading demo data

If you don't have FLEKS data to start with, you can download demo field data with the following:

In [ ]:
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/batsrus_data.tar.gz"
download_testfile(url, "data")

Example test particle data can be downloaded as follows:

In [ ]:
url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/test_particles_PBEG.tar.gz"
download_testfile(url, "data")

Example PIC particle data can be downloaded as follows:

In [ ]:
url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/3d_particle.tar.gz"
download_testfile(url, "data")

## Loading data

`flekspy.load` is the interface to read files of all formats. It returns a different object for different formats. IDL format data are processed into XArray data structures:

In [ ]:
file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)
ds

The coordinates can be accessed via

In [ ]:
ds.coords

The variables can be accessed via

In [ ]:
ds.var

and individual variables can be accessed through keys or properties:

In [ ]:
ds["p"]
ds.p

### Interpolating data

Variable interpolation is supported via [XArray](https://docs.xarray.dev/en/stable/user-guide/interpolation.html). For 1D data, simply provide the coordinate:

In [ ]:
ds["Rho"].interp(x=100)

For multidimensional interpolation, it can be easily extended as

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
ds["rhoS0"].interp(x=-28000.0, y =0.0)

If you want to interpolate for all variables at a single location,

In [ ]:
ds.interp(x=-28000.0, y =0.0)

## Visualizing fields

Thanks to XArray's Matplotlib wrapper, all Matplotlib's plotting functionalities are directly supported.

In [ ]:
import matplotlib.pyplot as plt

file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)

fig, axs = plt.subplots(2, 1, constrained_layout=True, sharex=True, sharey=True)
ds.p.plot(ax=axs[0])
ds.Bx.plot(ax=axs[1])
plt.show()

By default, XArray's 2D plotting functions map the first dimension to the vertical y-axis and the second dimension to the horizontal x-axis. To overwrite this default behavior, we can explicitly set the `x` and `y` arguments:

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
fig, axs = plt.subplots(3, 1, figsize=(10, 6), constrained_layout=True, sharex=True, sharey=True)
ds.Bx.plot.pcolormesh(ax=axs[0], x="x", y="y")
ds.By.plot.pcolormesh(ax=axs[1], x="x", y="y")
ds.Bz.plot.pcolormesh(ax=axs[2], x="x", y="y")
plt.show()

Native AMReX format can be read using YT:

In [ ]:
ds = flekspy.load("data/3d*amrex")

We can take a slice at a given location, which works both for IDL and AMReX data:

In [ ]:
dc = ds.get_slice("z", 0.5)
dc

and plot the selected variables on the slice with colored contours, streamlines, and contour lines:

In [ ]:
f, axes = dc.plot("By", pcolor=True)
dc.add_stream(axes[0], "Bx", "By", color="w")
dc.add_contour(axes[0], "Bx")

Users can also obtain the data of a 2D plane, and visualize it with matplotlib. A complete example is given below:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ds = flekspy.load("data/3*amrex")
dc = ds.get_slice("z", 0.5)

f, axes = plt.subplots(
    1, 2, figsize=(12, 4), constrained_layout=True, sharex=True, sharey=True
)

fields = ["By", "Bz"]
for ivar in range(2):
    v = dc.evaluate_expression(fields[ivar])
    vmin = v.min().v
    vmax = v.max().v
    ax = axes[ivar]
    ax.set_title(fields[ivar], fontsize=16)
    ax.set_ylabel("Y", fontsize=16)
    ax.set_xlabel("X", fontsize=16)
    c = ax.pcolormesh(dc.x.value, dc.y.value, np.array(v.T), cmap="turbo")
    cb = f.colorbar(c, ax=ax, pad=0.01)

    ax.set_xlim(np.min(dc.x.value), np.max(dc.x.value))
    ax.set_xlim(np.min(dc.y.value), np.max(dc.y.value))
    dc.add_stream(ax, "Bx", "By", density=0.5, color="w")

plt.show()

## Visualizing phase space distributions

In [ ]:
from flekspy.util import unit_one

data_file = "data/3d_*amrex"
ds = flekspy.load(data_file)

# Add a user defined field. See yt document for more information about derived field.
ds.add_field(
    ds.pvar("unit_one"),
    function=unit_one,
    sampling_type="particle",
    units="dimensionless",
)

x_field = "p_uy"
y_field = "p_uz"
# z_field = "unit_one"
z_field = "p_w"
xleft = [-0.016, -0.01, 0.0]
xright = [0.016, 0.01, 1.0]

zmin, zmax = 1e-5, 2e-3

## Select and plot the particles inside a box defined by xleft and xright
region = ds.box(xleft, xright)
pp = ds.plot_phase(
    x_field,
    y_field,
    z_field,
    region=region,
    unit_type="si",
    x_bins=100,
    y_bins=32,
    domain_size=(xleft[0], xright[0], xleft[1], xright[1]),
)

pp.set_cmap(pp.fields[0], "turbo")
pp.set_font(
    {
        "size": 34,
        "family": "DejaVu Sans",
    }
)

pp.set_zlim(pp.fields[0], zmin=zmin, zmax=zmax)

pp.set_xlabel(r"$V_y$")
pp.set_ylabel(r"$V_z$")
pp.set_colorbar_label(pp.fields[0], "weight")
str_title = (
    rf"x = [{xleft[0]:.1e}, {xright[0]:.1e}], y = [{xleft[1]:.1e}, {xright[1]:.1e}]"
)
pp.set_title(pp.fields[0], str_title)
pp.set_log(pp.fields[0], True)
pp.show()
# pp.save("test")
# pp.plots[("particles", z_field)].axes.xaxis.set_major_locator(
# ticker.MaxNLocator(4))
# pp.plots[("particles", z_field)].axes.yaxis.set_major_locator(
# ticker.MaxNLocator(4))

### Plot the location of particles that are inside a sphere

In [ ]:
center = [0, 0, 0]
radius = 1
# Object sphere is defined in yt/data_objects/selection_objects/spheroids.py
sp = ds.sphere(center, radius)
pp = ds.plot_particles(
    "p_x", "p_y", "p_w", region=sp, unit_type="si", x_bins=64, y_bins=64
)
pp.show()
# pp.save("test")

### Plot the velocity space of particles that are inside a sphere

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", "p_w", region=sp, unit_type="si", x_bins=64, y_bins=64
)

pp.set_cmap(pp.fields[0], "turbo")

pp.set_font(

    {

        "size": 34,

        "family": "DejaVu Sans",

    }
)

pp.show()

### Plot the location of particles that are inside a disk

In [ ]:
center = [0, 0, 0]
normal = [0, 0, 1]  # normal direction of the disk
radius = 0.5
height = 1.0
# Object sphere is defined in yt/data_objects/selection_objects/disk.py
disk = ds.disk(center, normal, radius, height)
pp = ds.plot_particles(
    "p_x", "p_y", "p_w", region=disk, unit_type="si", x_bins=64, y_bins=64
)
pp.show()

### Plot the velocity space of particles that are inside a disk

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", "p_w", region=disk, unit_type="si", x_bins=64, y_bins=64
)

pp.show()

Get the 2D array from the phase plot and reconstruct from scratch:

In [ ]:
for var_name in pp.profile.field_data:
    val = pp.profile.field_data[var_name]

x = pp.profile.x
y = pp.profile.y

plt.pcolormesh(x, y, val)
plt.show()

## Plotting Test Particle Trajectories

### Loading particle data

In [ ]:
from flekspy import FLEKSTP

tp = FLEKSTP("data/test_particles_PBEG", iSpecies=1)

### Obtaining particle IDs

Test particle IDs consists of a CPU index and a particle index attached to the CPU.

In [ ]:
tp.getIDs()

### Reading particle trajectory

In [ ]:
tp[0].collect()

### Getting initial location

In [ ]:
x = tp.read_initial_condition(tp.getIDs()[0])
print("time, X, Y, Z, Vx, Vy, Vz")
print(
    f"{x[0]:.2e}, {x[1]:.2e}, {x[2]:.2e}, {x[3]:.2e}, {x[4]:.2e}, {x[5]:.2e}, {x[6]:.2e}"
)

### Interpolate particles trajectory at selected times

In [ ]:
from flekspy.tp import interpolate_at_times

interpolate_at_times(tp[0], [1.0])

### Plotting trajectory

In [ ]:
tp.plot_trajectory(tp.getIDs()[0])
plt.show()

### Reading and visualizing all particle's location at a given snapshot

In [ ]:
ids, pData = tp.read_particles_at_time(0.0, doSave=False)
tp.plot_location(pData)
plt.show()

### Selecting particles starting in a region

In [ ]:
from flekspy.tp import Indices

def f_select(tp, pid):
    pData = tp.read_initial_condition(pid)
    inRegion = pData[Indices.X] > 0 and pData[Indices.Y] > 0
    return inRegion


pSelected = tp.select_particles(f_select)

### Saving trajectories to CSV

In [ ]:
tp.save_trajectory_to_csv(tp.getIDs()[0])

### Calculating drifts

With the $\mathbf{E}, \mathbf{B}$, and $\nabla \mathbf{B}$ saved along the trajectory, we can compute various drifts:
- Convection drift velocity: `tp.get_ExB_drift(pid)`
- Curvature drift velocity: `tp.get_curvature_drift(pid)`
- Gradient drift velocity: `tp.get_gradient_drift(pid)`
- Polarization drift velocity: `tp.get_polarization_drift(pid)`

The first adiabatic assumption can be checked by comparing the ratio between the gyroradius and the curvature length $r_L / r_c$ via `get_gyroradius_to_curvature_ratio`. If $r_L / r_c \ll 1$, then the gyromotion can be savely ignored.


The Betatron acceleration $\mu \partial B/\partial t$ can be indirectly computed with the knowledge of total derivative $\mathrm{d}B/\mathrm{d}t$ and $\mathbf{v}\nabla B$ along the trajectory:

In [ ]:
pid = tp.getIDs()[0]
pt = tp[pid]
mu = tp.get_first_adiabatic_invariant(pt)
pt = tp.get_betatron_acceleration(pt, mu)

To analyze a specific drift:

In [ ]:
tp.analyze_drift(pid, "gradient")

By combining all these together, we come up with time series of various terms:

In [ ]:
tp.analyze_drifts(pid)

We can the track the energy change from drift and EM field variations:

In [ ]:
from flekspy.tp import plot_integrated_energy

df = tp.integrate_drift_accelerations(tp.getIDs()[0])
plot_integrated_energy(df)

Alternatively, the energy change can be decomposed into three terms: the parallel acceleration, Betatron acceleration, and the Fermi acceleration:

In [ ]:
df = tp.get_energy_change_guiding_center(tp.getIDs()[0])

df